# Introduction

This notebook supports the recreation of figures from "A critical reflection on attempts to machine-learn materials synthesis insights from 
text-mined literature recipes" by Wenhao Sun and Nicholas David. Each section can be run independently of all other sections. All figures will be saved to the "figs" directory for viewing.

# Get text-mined recipes data

In [2]:
import os
import requests
import zipfile
import lzma
import json

os.makedirs("figs", exist_ok=True)

In [2]:
def get_content(url: str, path: str):
    '''
    Pulls content from a url and saves it to path

    Args:
        url (str): The URL to pull data from
        path (str): The path to save the pulled data

    Returns:
        None
    '''
    if not os.path.exists(path):
        if '/' in path:
            # Local path where the file will be saved
            local_dir = ''.join(path.split('/')[0:-1])
            local_path = path.split('/')[-1]
            # Create the directory if it doesn't exist
            os.makedirs(local_dir, exist_ok=True)

        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Write the content to the local file
            with open(path, 'wb') as file:
                file.write(response.content)
            print(f"File downloaded successfully and saved to {path}")
        else:
            print(f"Failed to download file. HTTP Status Code: {response.status_code}")
    else:
        print(f'File already exists with path {path}')
    
def unpack_content(fp: str, save_dir: str):
    '''
    Unpacks the downloaded content

    Args:
        fp (str): Filepath to the downloaded content
        save_dir (str): Directory to save the extracted content
    
    Returns:
        None
    '''
    fe = fp.split('.')[-1]
    if fe == 'zip':
        with zipfile.ZipFile(fp, 'r') as zip_ref:
            zip_ref.extractall(save_dir)
            print(f'Successfully unpacked {fp} to {save_dir}')
    elif fe == 'xz':
        save_path = fp[0:-3]
        with lzma.open(fp) as xz_file:
           json_bytes = xz_file.read()
           stri = json_bytes.decode('utf-8')
           data = json.loads(stri)
        with open(save_path, 'w') as f:
            json.dump(data, f, indent=1)
            print(f'Successfully unpacked {fp} to {save_dir}')
    else:
        print(f"Unsupported file type {fe}")
        raise TypeError


In [3]:
solutions_github = 'https://github.com/CederGroupHub/text-mined-solution-synthesis_public/raw/main/solution-synthesis_dataset_2021-8-5.json.zip'
solid_state_github = 'https://github.com/CederGroupHub/text-mined-synthesis_public/raw/master/solid-state_dataset_20200713.json.xz'
get_content(solutions_github, 'data/solution-synthesis_dataset_2021-8-5.json.zip')
get_content(solid_state_github, 'data/solid-state_dataset_20200713.json.xz')
unpack_content('data/solution-synthesis_dataset_2021-8-5.json.zip', 'data')
unpack_content('data/solid-state_dataset_20200713.json.xz', 'data')

File already exists with path data/solution-synthesis_dataset_2021-8-5.json.zip
File already exists with path data/solid-state_dataset_20200713.json.xz
Successfully unpacked data/solution-synthesis_dataset_2021-8-5.json.zip to data
Successfully unpacked data/solid-state_dataset_20200713.json.xz to data


# Figure 1: Pareto-principle of target chemical systems

This section provides the code needed to reproduce Figure 1. The workflow loads and extracts target chemical formulas from the text-mined recipes dataset and organizes them by the frequency of their chemical system. Lastly, the data is plotted using the pareto-principle.

In [4]:
import pymatgen.core as pmg
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from collections import defaultdict
import json

In [5]:
def get_recipe_dict(synthesis_paths: dict) -> dict:
    '''
    Returns recipe dict from synthesis paths

    Args:
        synthesis_paths (dict): Name, paths to synthesis recipe data

    Returns:
        recipe_dict (dict): Name, recipes dictionary
    '''
    for index, (name, sp) in enumerate(synthesis_paths.items()):
        with open(sp) as f:
            recipes = json.load(f)
            if name == "Solid-state":
                solid_recipes = recipes['reactions']
            else:
                sol_recipes = recipes
    recipe_dict = {"Solid-state": solid_recipes,
                   "Solution": sol_recipes}
    return recipe_dict

def ensure_O_at_end(s: str):
    '''
    Places oxygen ('O') as the last element in the chemsys string

    Args:
        s (string): chemsys string
    Returns:
        ret (string): oxygen at end chemsys string
    '''
    if 'O' in s.split("-"):
        splits = s.split("-")
    else:
        return s
    ret = ""
    for i, c in enumerate(splits):
        if c == "O" and i != len(splits) - 1:
            continue
        elif c != "O":
            ret += f"{c}-"
    ret += 'O'
    return ret

def pareto_viz(recipe_dict: dict, plot_title: str, save_path: str):
    '''
    Generates and saves the pareto-principle visualization to the specified save_path.
    Additionally, saves a list of the most frequent materials

    Args:
        recipe_dict (dict): Name and recipes list for solid-state and solution recipes
        plot_title (str): Title of the plot
        save_path (str): Path to save the generated visualization
    '''
    # Form the data for plotting
    data = {}
    for i, (name, recipes) in enumerate(recipe_dict.items()):
        freq_dict = defaultdict(int)
        freq_dict_formula = defaultdict(int)
        for recipe in recipes:
            target_formula = recipe['target']['material_formula']
            # ignores some target compositions that are poorly formed
            try:
                target_comp = pmg.Composition(target_formula)
            except:
                target_comp = ""
            if target_comp != "":
                freq_dict[str(target_comp.chemical_system)] += 1
                freq_dict_formula[str(target_comp)] += 1
        sorted_freq_dict = dict(sorted(freq_dict.items(), key=lambda item: item[1], reverse=True))
        cumsum = np.cumsum(list(sorted_freq_dict.values()))
        percents = cumsum*100/np.sum(list(sorted_freq_dict.values()))
        mats = list(sorted_freq_dict.keys())
        x_plot = list(range(len(mats)))
        inds = []
        mat_inds = []
        cum_percs = [20, 40, 60, 80, 100]
        for x in cum_percs:
            ind = np.abs(percents - x).argmin()
            inds.append(ind)
            mat_inds.append(x_plot[ind])
        if name == "Solid-state":
            data["Solid"] = {"mats": mats, "mat_inds": mat_inds, "percs": percents, "inds": inds, "x": x_plot}
            top_solid = list(sorted_freq_dict.keys())[0:5]
            top_solid = [ensure_O_at_end(s) for s in top_solid]
        else:
            data["Sol"] = {"mats": mats, "mat_inds": mat_inds, "percs": percents, "inds": inds, "x": x_plot}
            top_sol = list(sorted_freq_dict.keys())[0:5]
            top_sol = [ensure_O_at_end(s) for s in top_sol]
    # Make the figure. Much is just placement of chart annotations
    fig = go.Figure()
    inds = data['Solid']["inds"]
    fig.add_trace(go.Scatter(x=data["Solid"]["x"], y=data["Solid"]["percs"], mode='lines', 
                             name='Solid-state', line=dict(color='rgb(29, 105, 150)'), showlegend=False))
    annotations = []
    for i in range(len(cum_percs)):
        if i == len(cum_percs) - 1:
            text = f"{len(data["Solid"]["x"])} unique"
        elif i == 0:
            text = f"Top {inds[i] + 1}"
        else:
            text = f"{inds[i] + 1}"
        if i == 0:
            ax = 30
            ay = 0
        elif i == len(cum_percs) - 1:
            ax = 0
            ay = 20
        elif i == len(cum_percs) - 2:
            ax = 40
            ay = 0
        else:
            ay = 0
            ax = 20
        annotations.append(dict(x=data["Solid"]["mat_inds"][i], y=cum_percs[i], xref='x', yref='y', 
                                text=text, showarrow=True, font=dict(color='rgb(29, 105, 150)'), ax=ax, ay=ay))
    inds = data['Sol']["inds"]
    fig.add_trace(go.Scatter(x=data["Sol"]["x"], y=data["Sol"]["percs"], mode='lines', 
                             showlegend=False, name='Solution', line=dict(color='rgb(15, 133, 84)')))
    for i in range(len(cum_percs)):
        if i == len(cum_percs) - 1:
            text = f"{len(data["Sol"]["x"])} unique"
        elif i == 0:
            text = f"Top {inds[i] + 1}"
        else:
            text = f"{inds[i] + 1}"
        if i == 0:
            ax = -25
            ay = 0
        elif i == len(cum_percs) - 1:
            ax = 0
            ay = -13
        else:
            ax = -20
            ay = 0
        annotations.append(dict(x=data["Sol"]["mat_inds"][i], y=cum_percs[i], xref='x', 
                                yref='y', text=text, showarrow=True, font=dict(color='rgb(15, 133, 84)'), ax=ax, ay=ay))
    xaxis_title="Chemical System"
    largest_x = len(data["Solid"]["x"]) if len(data["Solid"]["x"]) > len(data["Sol"]["x"]) else len(data["Sol"]["x"])
    xmin = -largest_x//7
    xmax = largest_x
    solid_annot_text = "Top 5 Solid-state Systems\n"
    for chemsys in top_solid:
        solid_annot_text += f"{chemsys}\n"
    sol_annot_text = "Top 5 Solution Systems\n"
    for chemsys in top_sol:
        sol_annot_text += f"{chemsys}\n"
    for i, line in enumerate(solid_annot_text.splitlines()):
        annotations.append(dict(x=(largest_x*2)//5, y=cum_percs[-1]//2 - i*5, xref='x', yref='y',
                       text=line, showarrow=False, font=dict(color='rgb(29, 105, 150)')))
    for i, line in enumerate(sol_annot_text.splitlines()):
        annotations.append(dict(x=(largest_x*3)//4, y=cum_percs[-1]//2 - i*5, xref='x', yref='y',
                       text=line, showarrow=False, font=dict(color='rgb(15, 133, 84)')))
    for i, perc in enumerate(cum_percs):
        if perc == 100:
            fig.add_shape(type="line", x0=xmin, y0=perc, x1=data["Solid"]["mat_inds"][i], y1=perc, line=dict(color="grey", width=2, dash="dash"))
    fig.update_layout(title=plot_title,
                      font_family="DejaVu Sans",
                      annotations=annotations,
                      xaxis_title=xaxis_title,
                      xaxis=dict(tickfont=dict(color="rgb(0,0,0)", size=1), range=[xmin, xmax], color="rgb(0,0,0)", title=dict(standoff=0)),
                      title_x=0.5,
                      yaxis=dict(title='Cumulative Percentage (%)',
                                  side='left',
                                  color="rgb(0,0,0)"),
                      paper_bgcolor='white',
                      plot_bgcolor='white'
    )
    #fig.write_image(f"{save_path}.svg")
    fig.write_image(f"{save_path}.png")
    fig.show()

In [6]:
synthesis_paths = {"Solid-state": "data/solid-state_dataset_20200713.json",
                   "Solution": "data/solution-synthesis_dataset_2021-8-5.json"
                   }
recipe_dict = get_recipe_dict(synthesis_paths)
plot_title = "Pareto-principle of Target Material Chemical Systems"
save_path = "figs/pareto"
pareto_viz(recipe_dict, plot_title, save_path)

# Figure 2: Sulfides heatmap

This section provides the code needed to reproduce Figure 2. Since the Materials Platform for Data Science (MPDS) data is not open-source, a file containing just the grid points for MPDS is provided. If you have an MPDS key, you can supply it below and construct the figure directly.

In [78]:
import os
from mpds_client import MPDSDataRetrieval
import pandas as pd
import numpy as np
import pymatgen.core as pmg
from typing import Tuple, List
from mp_api.client import MPRester
from monty.serialization import loadfn, dumpfn
import copy
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
import ast

In [39]:
# Supply you MP and MPDS keys below"
MP_KEY = ""
MPDS_KEY = ""
os.environ["MPDS_KEY"] = MPDS_KEY

## Retrieval of sulfur compounds from MP, MPDS, and text-mined recipes

Before construction of the membership grid, we will first need to gather all the available ternary sulfides available within in each materials database. IF YOU DO NOT HAVE MPDS ACCESS, SKIP TO "Construct the heatmap"

In [40]:
# Helper function to parse pymatgen compositions for determining materials classes
# (e.g. ternary oxides)

def comp_matcher(x, search):
    '''
    Decides whether a composition falls under the search chemsys

    Args:
        x (pmg.Composition): The test composition
        search (str): The search chemsys. * denotes wildcard matching. 
        (e.g. '*-*-S for ternary sulfur containing materials)
    
    Returns:
        bool: True if the composition falls under the search chemsys
    '''
    s1 = search.split('-')
    nots = []
    s = []
    for c in s1:
        if c[0] == '!':
            nots.append(pmg.Element(c[1:]))
        else:
            s.append(c)
    l = len(s)
    wildcard = '*'
    if len(x.elements) == len(s):
        for c in s:
            if c != wildcard:
                e = pmg.Element(c)
                if e not in x.elements or e in nots:
                    return False
    else:
        return False
    return True

### MPDS retrieval

In [50]:
def get_entries(save_path: str) -> pd.DataFrame:
    '''
    Retrieves all ternary structure files from MPDS. Saves to excel or reads from
    if file already exists.

    Args:
        save_path (str): Path to save the query
    
    Returns:
        df (pd.DataFrame): The data
    '''
    if os.path.exists(save_path):
        df = pd.read_excel(save_path)
    else:
        client = MPDSDataRetrieval(MPDS_KEY, dtype=1)
        client.maxnpages = 500
        fields = {'S': [
            'phase', 'phase_id', 'chemical_formula', 'prototype', 'sg_n', 'sg_hm', 'cell_abc', 
            'basis_noneq', 'symops', 'els_noneq', 'labels_noneq', 'occs_noneq', 'periodic_nums_noneq', 
            'linus_pauling_nums_noneq', 'wyck_noneq', 'ssym_noneq', 'entry', 'version', 'object_repr', 
            'object_type', 'reference', 'density', 'volume', 'r_factor', 'condition', 'preparation', 'comments'
        ]}
        fields = {'S': [
            'chemical_formula'
        ]}
        response = client.get_dataframe(search={'classes': 'ternary'}, fields=fields, columns=fields['S'])
        df = response.dropna(axis=0, how='all')
        df.to_excel(save_path)
    return df

def form_composition(chemical_formula) -> str | pmg.Composition:
    '''
    Converts the chemical formula into a pymatgen composition if well formed

    Args:
        chemical_formula (str): The chemical formula from MPDS
    
    Returns:
        comp (str | pmg.Composition): The converted chemical formula
    '''
    chemical_formula = chemical_formula.replace('[', '(')
    chemical_formula = chemical_formula.replace(']', ')')
    try:
        comp = pmg.Composition(chemical_formula)
    except ValueError:
        comp = ""
    return comp

def get_mpds_comps(fp: str, search: str) -> set:
    '''
    Returns the compositions from MPDS according to the search chemsys 

    Args:
        fp (str): The filepath to the MPDS returned data
        search (str): The search chemsys
    
    Returns:
        ret_set (set): The returned set of compositions
    '''
    mpds_df = get_entries(fp)
    mpds_df['pmg_comp'] = mpds_df['chemical_formula'].apply(form_composition)
    # Ensure that all match the search chemsys
    ret_set = set()
    for x in mpds_df['pmg_comp']:
        if isinstance(x, pmg.Composition) and comp_matcher(x, search):
            ret_set.add(x)
    return ret_set

In [51]:
mpds_TS_set = get_mpds_comps("data/mpds_ternaries.xlsx", '*-*-S')

	99% of step 1 from 1 Got 369224 hits


### MP Retrieval

You can either choose to retrieval all experimentally confirmed compounds or just the sulfur containing ternary compounds

In [55]:
def get_all_experimental(save_path: str, chemsys: str='') -> list:
    '''
    Retrieves all the experimental compounds from Materials Project. Can also filter
    by chemsys if supplied

    Args:
        save_path (str): The path to save the response
        chemsys (str): An optional chemsys to filter entries
    
    Returns:
        docs (list): A list of returned matching documents
    '''
    if not os.path.exists(save_path):
        with MPRester(MP_KEY) as mpr:
            if chemsys != '':
                docs = mpr.summary.search(theoretical=False, fields=["composition"], chemsys=chemsys)
            else:
                docs = mpr.summary.search(theoretical=False, fields=["composition"])
        dumpfn(docs, save_path)
    docs = loadfn(save_path)
    return docs


In [56]:
def get_mp_comps(fp: str, search: str) -> set:
    '''
    Returns the compositions for ternary oxides from mp

    Args:
        fp (str): The filepath to the mp returned data

    Returns:
        ret_set (set): The set of all compliant compositions
    '''
    docs = get_all_experimental(fp, search)
    ret_set = set()
    for x in docs:
        comp = pmg.Composition(x['composition'])
        if isinstance(comp, pmg.Composition) and (comp_matcher(comp, search) or search == ''):
            ret_set.add(comp)
    return ret_set

In [57]:
mp_TS_set = get_mp_comps('data/exp_mp_TS.json', search='*-*-S')

### TMR retrieval

In [58]:
# Helper function to generate a simplified recipe dictionary
def get_recipe_dict(synthesis_paths: dict) -> dict:
    '''
    Returns recipe dict from synthesis paths

    Args:
        synthesis_paths (dict): Name, paths to synthesis recipe data

    Returns:
        recipe_dict (dict): Name, recipes dictionary
    '''
    for index, (name, sp) in enumerate(synthesis_paths.items()):
        with open(sp) as f:
            recipes = json.load(f)
            if name == "Solid-state":
                solid_recipes = recipes['reactions']
            else:
                sol_recipes = recipes
    recipe_dict = {"Solid-state": solid_recipes,
                   "Solution": sol_recipes}
    return recipe_dict

In [59]:
# Helper functions to generate compositions from complex cases that contain
# possible substitutions
def generate_compositions(compositions: List[dict], amounts_vars: dict, elements_vars: dict) -> Tuple[List[pmg.Composition], bool, bool]:
    '''
    Generates all compositions given a composition entry by substitutions of variable amounts and elements

    Args:
        compositions (list[dict]): A list of composition dict entries from the recipes
        amounts_vars (dict): An amount vars dict entry from the recipes
        elements_vars (dict): An element vars dict entry from the recipes

    Returns:
        ret_all_target_comps (List(pmg.Composition)): All possible target compositions
        name_error (bool): Whether there was a variable substitution error
        value_error (bool): Whether there was a value (numeric) substituion error
    '''
    all_target_comps = []
    all_elem_sub_target_comps = []
    name_error = False
    value_error = False
    for comp in compositions:
        # Do the elemental substitutions first
        new_comps = []
        just_added = {}
        # create combinations of the possible elem subs
        combos = [dict(zip(elements_vars.keys(), values)) for values in product(*elements_vars.values())]
        for combo in combos:
            new_comp = copy.deepcopy(comp['elements'])
            for tmp_elem, elem in combo.items():
                try:
                    new_comp[elem] = comp['elements'][tmp_elem]
                    del new_comp[tmp_elem]
                except KeyError:
                    pass
            if new_comp != just_added:
                new_comps.append(new_comp)
                just_added = new_comp
        all_elem_sub_target_comps.extend(new_comps)
    # now do the amounts_vars subs
    # first we format the amounts_vars into key:list[value] pairs
    amounts = {}
    for letter in amounts_vars:
        if len(amounts_vars[letter]) > 0:
            amounts[letter] = amounts_vars[letter]['values']
    # Now amounts = {'x': [0, 0.2, 0.4], 'y': [0, 0.3, 0.5], ...}
    for comp in all_elem_sub_target_comps:
        new_comps = []
        # create mixtures of amounts
        combos = [dict(zip(amounts.keys(), values)) for values in product(*amounts.values())]
        if len(combos) == 0:
            all_target_comps.append(comp)
        for combo in combos:
            new_comp = copy.deepcopy(comp)
            for elem, val in new_comp.items():
                for letter, amt in combo.items():
                    new_comp[elem] = new_comp[elem].replace(letter, str(amt))
            new_comps.append(new_comp)
        all_target_comps.extend(new_comps)
    ret_all_target_comps = []
    for targ_comp in all_target_comps:
        successful_eval = True
        for elem, val in targ_comp.items():
            try:
                targ_comp[elem] = float(eval(val))
            except NameError:
                name_error = True
                successful_eval = False
        if successful_eval:
            try:
                ret_all_target_comps.append(pmg.Composition(targ_comp))
            except ValueError:
                value_error = True
    return ret_all_target_comps, name_error, value_error

def generate_target_compositions(entry: dict) -> Tuple[List[pmg.Composition], bool, bool]:
    '''
    Generates all possible target compositons given any possible variable
    amounts of dopants and elemntal subsitutions.
    
    Args:
        entry: Synthesis recipe
    
    Returns:
        ret_all_target_comps (List(pmg.Composition)): All possible target compositions
        name_error (bool): Whether there was a variable substitution error
        value_error (bool): Whether there was a value (numeric) substituion error
    '''
    compositions = entry['target']['composition']
    amounts_vars = entry['target']['amounts_vars']
    elements_vars = entry['target']['elements_vars']
    return generate_compositions(compositions, amounts_vars, elements_vars)

In [60]:
def get_tmr_comps(recipe_dict: dict, search: str):
    '''
    Retrieves all compositions from the TMR datasets that match search

    Args:
        recipe_dict (dict): A dictionary containing the two TMR datasets

    Returns:
        tern_set (set): Compositions 
    '''
    ret_set = set()
    for name, recipes in recipe_dict.items():
        for recipe in recipes:
            comps, _, _ = generate_target_compositions(recipe)
            elements = []
            for comp in comps:
                if isinstance(comp, pmg.Composition) and comp_matcher(comp, search):
                    # only add if elements are unique
                    if elements != comp.elements:
                        ret_set.add(comp)
                        elements = comp.elements
    return ret_set

In [61]:
synthesis_paths = {"Solid-state": "data/solid-state_dataset_20200713.json",
                   "Solution": "data/solution-synthesis_dataset_2021-8-5.json"
                   }
recipe_dict = get_recipe_dict(synthesis_paths)
tmr_TS_set = get_tmr_comps(recipe_dict, '*-*-S')

### Combining the compositions

In [62]:
def tern_set_to_chemsys(tern_set: set[pmg.Composition]) -> set[str]:
    '''
    Takes the set chemical compositions and returns a set of their chemsys

    Args:
        tern_set (set[pmg.Composition]): The pulled compositions
    
    Returns:
        ret_set (set[str]): A reduced set of the compositions chemical systems
    '''
    ret_set = set()
    for x in tern_set:
        ret_set.add(x.chemical_system)
    return ret_set

def gen_combined_df(tmr_terns: set[pmg.Composition], mp_terns: set[pmg.Composition], mpds_terns: set[pmg.Composition]) -> pd.DataFrame:
    '''
    Combines all the pulled compositions into a single dataframe of chemical
    systems and dataset memeberships

    Args:
        tmr_terns (set): The compositions from the text-mined recipes
        mp_terns (set): The compositions from Materials Project
        mpds_terns (set): The compositions from MPDS
    
    Returns:
        df (pd.DataFrame): A dataframe containing the chemsys and it's membership from each dataset
    '''
    chemsys_tmr = tern_set_to_chemsys(tmr_terns)
    chemsys_mp = tern_set_to_chemsys(mp_terns)
    chemsys_mpds = tern_set_to_chemsys(mpds_terns)
    combined_set = chemsys_tmr.union(chemsys_mp, chemsys_mpds)
    combined_list = list(combined_set)
    memberships = []
    for chemsys in combined_set:
        membership = []
        if chemsys in chemsys_tmr:
            membership.append("TMR")
        if chemsys in chemsys_mp:
            membership.append("MP")
        if chemsys in chemsys_mpds:
            membership.append("MPDS")
        memberships.append(tuple(membership))
    df = pd.DataFrame({'chemsys': combined_list, "membership": memberships})
    return df

In [68]:
grid_data_path = 'data/grid_members.xlsx'
dfg = gen_combined_df(tmr_TS_set, mp_TS_set, mpds_TS_set)
dfg.to_excel(grid_data_path, index=False)

## Construct the heatmap

Start here if you want to skip the data acquisition if, for example, you do not have MPDS API access.

In [237]:
# Some helpers
def gen_binary_from_chemsys(chemsys: str, removal: list[str]) -> list[str]:
    '''
    Returns the two plotting elements using the input chemsys and a list of
    removal elements
    
    Args:
        chemsys (str): The entry's chemsys
        removal (list[str]): Elements to remove, reducing it to binary
    
    Return:
        elems (list[str]): Two elements to plot
    '''
    elems = chemsys.split('-')
    for e in removal:
        if e in elems:
            elems.remove(e)
    return elems

def dendro(data: np.ndarray, labels: list[str], save_path: str) -> list[str]:
    '''
    Makes the dendrogram using hierarchical clustering and yields the reordering of axis elements

    Args:
        data (np.ndarray): The membership matrix
        labels (list[str]): The starting order of axis labels
        save_path (str): The path to save the dendrogram
    
    Returns:
        lab_ret (list[str]): The reordered list of axis elements
    '''
    # Calculate Euclidean distance matrix
    distance_matrix = np.zeros((data.shape[0], data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(i+1, data.shape[0]):
            distance_matrix[i, j] = np.linalg.norm(data[i] - data[j])

    # Apply hierarchical clustering
    linkage_matrix = linkage(distance_matrix, method='average', optimal_ordering=True)

    # Plot dendrogram
    plt.figure(figsize=(11, 9))
    # print(linkage_matrix)
    ctp = 0.4
    dendrogram(linkage_matrix, labels=labels, color_threshold=ctp*np.max(linkage_matrix[:, 2]), leaf_rotation=90)
    locs, labs = plt.xticks()
    plt.ylabel('Distance')
    plt.xlabel('Data points')
    plt.ylim((0, 1000))
    plt.axis('off')
    plt.savefig(f"{save_path}_dendro_{ctp}.png")
    #plt.savefig(f"{save_path}_dendro_{ctp}.svg")
    lab_ret = [x._text for x in labs]
    plt.close()
    return lab_ret

In [85]:
def grid_plot(dfg: pd.DataFrame, save_path: str, removal: list[str]=['O'], add_petifor: list[str]=[]):
    '''
    Makes and saves the grid visualization. After effects and joining of the
    dendrogram is done by hand.

    Args:
        dfg (pd.DataFrame): The grid dataframe will all membership resolved
        save_path (str): The path to save the visualization
        removal (list): Elements to remove from the grid. Typically the third (or
            fourth) element in the chemsys.
        add_petifor (list): Elements to explcitly add to the petifor after filtering
    '''
    petifor_og = ['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn', 'Fr', 'Cs', 'Rb', 'K', 'Na', 
           'Li', 'Ra', 'Ba', 'Sr', 'Ca', 'Eu', 'Yb', 'Lu', 'Tm', 'Y', 'Er', 
           'Ho', 'Dy', 'Tb', 'Gd', 'Sm', 'Pm', 'Nd', 'Pr', 'Ce', 'La', 'Ac', 
           'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 
           'Md', 'No', 'Lr', 'Sc', 'Zr', 'Hf', 'Ti', 'Ta', 'Nb', 'V', 'Cr', 
           'Mo', 'W', 'Re', 'Tc', 'Os', 'Ru', 'Ir', 'Rh', 'Pt', 'Pd', 'Au', 
           'Ag', 'Cu', 'Ni', 'Co', 'Fe', 'Mn', 'Mg', 'Zn', 'Cd', 'Hg', 'Be', 
           'Al', 'Ga', 'In', 'Tl', 'Pb', 'Sn', 'Ge', 'Si', 'B', 'C', 'N', 'P', 
           'As', 'Sb', 'Bi', 'Po', 'Te', 'Se', 'S', 'O', 'At', 'I', 'Br', 'Cl', 
           'F', 'H']

    petifor = []
    # filter out some chemistries
    for x in petifor_og:
        e = pmg.Element(x)
        if e.is_metal and not e.is_actinoid and not e.is_lanthanoid and x != 'Ra':
            petifor.append(x)
    palette = sns.color_palette("tab10")
    color_to_num = {"Green": 3, "Blue": 2, "Red": 1, "whitesmoke": 0}
    num_to_color = {0: "whitesmoke", 1: palette[3], 2: palette[0], 3: palette[2]}
    def color_logic(members: tuple) -> str:
        if ('TMR' in members and 'MP' in members and 'MPDS' in members) or \
        ('TMR' in members and 'MPDS' in members) or ('TMR' in members and 'MP' in members) or \
            ('TMR' in members):
            return 'Green'
        elif ('MP' in members and 'MPDS' in members) or 'MP' in members:
            return 'Blue'
        elif 'MPDS' in members:
            return 'Red'
        return None
    dfg['color'] = dfg['membership'].apply(color_logic)
    dfg['num'] = dfg['color'].apply(lambda x: color_to_num[x])
    unique_elems = set()
    unique_binaries = defaultdict(int)
    for i, row in dfg.iterrows():
        chemsys, membership, color, num = row
        elems = gen_binary_from_chemsys(chemsys, removal)
        assert(len(elems) == 2)
        unique_binaries[f'{elems[0]}-{elems[1]}'] = num
        unique_binaries[f'{elems[1]}-{elems[0]}'] = num
        for elem in elems:
            unique_elems.add(elem)
    mod_petifor = copy.copy(petifor)
    for e in add_petifor:
        if e not in mod_petifor:
            mod_petifor.append(e)
    for elem in petifor:
        if elem not in unique_elems and elem in mod_petifor:
            mod_petifor.remove(elem)
    matrix = np.zeros((len(mod_petifor), len(mod_petifor)))
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i != j:
                e1 = mod_petifor[i]
                e2 = mod_petifor[j]
                matrix[i, j] = unique_binaries[f'{e1}-{e2}']
    new_pet = dendro(matrix, mod_petifor, save_path)
    mod_petifor = new_pet
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i != j:
                e1 = mod_petifor[i]
                e2 = mod_petifor[j]
                matrix[i, j] = unique_binaries[f'{e1}-{e2}']
    total = 0
    rows, cols = matrix.shape

    # Create a matrix of -1s with the same shape
    result_matrix = -np.ones((rows, cols))

    # Set elements inside the lower triangle to their original values
    result_matrix[np.tril_indices(rows)] = np.tril(matrix)[np.tril_indices(rows)]
    # set diagonal to -1
    for i in range(result_matrix.shape[0]):
        result_matrix[i, i] = -1
    
    for i in range(len(color_to_num)):
        # Get the lower triangle of the matrix
        # Count the occurrences of 2 in the lower triangle
        count = np.count_nonzero(result_matrix == i)
        total += count
    special_num_to_color = {0: "white",
                            1: "red",
                            2: "blue",
                            3: "green"}
    for i in range(len(color_to_num)):
        # Get the lower triangle of the matrix
        # Count the occurrences of 2 in the lower triangle
        count = np.count_nonzero(result_matrix == i)
        print(f"Number of {i}s ({special_num_to_color[i]}): {count} ({np.round((count/total)*100, decimals=1)} %)")
    print(f"Total number of squares: {total}")
    mask = np.triu(np.ones_like(matrix, dtype=bool))
    sns.set_theme(style="white")
    sns.color_palette("pastel")

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))
    plt.xlabel("Element One")
    plt.ylabel("Element Two")
    plt.xticks(fontsize=7)
    plt.yticks(fontsize=7)
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(matrix, mask=mask, cmap=[num_to_color[i] for i in range(len(num_to_color))],
                square=True, linewidths=.5, cbar_kws={"shrink": .5},
                xticklabels=mod_petifor, yticklabels=mod_petifor)
    plt.savefig(f"{save_path}.png")
    #plt.savefig(f"{save_path}.svg")
    plt.close()

This will construct the heatmap and dendrogram in separate files.

In [84]:
search = '*-*-S'
dfg = pd.read_excel(grid_data_path, converters={'membership':ast.literal_eval})
removal = []
save_path = "figs/grid_sulfur"
for e in search.split('-'):
    if e != '*':
        removal.append(e)
grid_plot(dfg, save_path, removal)

Number of 0s (white): 503 (48.6 %)
Number of 1s (red): 143 (13.8 %)
Number of 2s (blue): 317 (30.6 %)
Number of 3s (green): 72 (7.0 %)
Total number of squares: 1035


# Figure 3: YBCO

The figure in the manuscript uses a combination of data derived and hand curated YBCO recipes from published literature. The procedure below captures all the YBCO recipes from the text-mined recipes. The excel "ybco_curated.xlsx" contains the hand curated dataset used.

## Get YBCO recipes details

In [219]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from habanero import Crossref
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [220]:
def get_recipe_dict(synthesis_paths: dict) -> dict:
    '''
    Returns recipe dict from synthesis paths

    Args:
        synthesis_paths (dict): Name, paths to synthesis recipe data

    Returns:
        recipe_dict (dict): Name, recipes dictionary
    '''
    for index, (name, sp) in enumerate(synthesis_paths.items()):
        with open(sp) as f:
            recipes = json.load(f)
            if name == "Solid-state":
                solid_recipes = recipes['reactions']
            else:
                sol_recipes = recipes
    recipe_dict = {"Solid-state": solid_recipes,
                   "Solution": sol_recipes}
    return recipe_dict



In [221]:
def get_ybco_entries(recipe_dict: dict) -> pd.DataFrame:
    '''
    Retrives all the YBCO entries from the text-mined recipes

    Args:
        recipe_dict (dict): The Solid-state and solution recipes by name, dataset
    
    Returns:

    '''
    entries = []
    for name, recipes in recipe_dict.items():
        for recipe in recipes:
            target = recipe['target']
            compositions = target['composition']
            found = False
            for comp in compositions:
                if all(x in comp['elements'].keys() for x in ['Y', 'Ba', 'Cu', 'O']):
                    found = True
                    if len(comp['elements']) == 4:
                        tag = 'Y-Ba-Cu-O'
                    elif len(comp['elements']) > 4:
                        tag = 'Y-Ba-Cu-O-*'
                    else:
                        tag = None
            if found:
                doi = recipe['doi']
                target_formula = target['material_formula']
                time, temp = None, None
                temps = []
                times = []
                precursors = []
                for operation in recipe['operations']:
                    if operation['type'] == 'HeatingOperation':
                        conditions = operation['conditions']
                        if conditions['heating_temperature'] != None:
                            for htemp in conditions['heating_temperature']:
                                # if htemp['units'][-1] != 'K' or htemp['units'][-1] == 'C':
                                #     print(htemp['units'])
                                assert(htemp['units'][-1] == 'K' or htemp['units'][-1] == 'C')
                                if htemp['units'][-1] == 'K':
                                    temps.append(htemp['max_value']+273)
                                else:
                                    temps.append(htemp['max_value'])
                        if conditions['heating_time'] != None:
                            for htime in conditions['heating_time']:
                                time_units = htime['units']
                                ttime = htime['max_value']
                                assert(time_units == 'min' or time_units == 'minutes' or time_units == 'h' or 
                                    time_units == 'day' or time_units == 'd' or time_units == 'hour' or 
                                    time_units == 'hours' or time_units == 'hrs' or time_units == 'hr')
                                if time_units == 'min' or time_units == 'minutes':
                                    ttime /=60
                                elif time_units == 'day' or time_units == 'd':
                                    ttime /= 24
                                times.append(ttime)
                for precursor in recipe['precursors']:
                    precursors.append(precursor['material_formula'])
                if len(times) > 0:
                    time = sum(times)
                else:
                    time = None
                if len(temps) > 0:
                    temp = max(temps)
                else:
                    temp = None
                entries.append({'target': target_formula, 'precursors': ";".join(precursors),
                                'T (C)': temp, 't (hr)': time, 'tag': tag, 'doi': doi})
    df = pd.DataFrame(entries)
    df.to_excel("data/ybco_pulled.xlsx", index=False)
    return df

In [222]:
synthesis_paths = {"Solid-state": "data/solid-state_dataset_20200713.json",
                   "Solution": "data/solution-synthesis_dataset_2021-8-5.json"
                   }
recipe_dict = get_recipe_dict(synthesis_paths)
get_ybco_entries(recipe_dict)

,target,precursors,T (C),t (hr),tag,doi
0,YxGd1-xBa2Cu3O7,CuO;BaCO3;Y2O3;Gd2O3,950.0,58.000,Y-Ba-Cu-O-*,10.1016/S0921-4534(02)01441-7
1,Ba2YCu3O6,Y2O3;CuO;BaO,1470.0,NaN,Y-Ba-Cu-O,10.1016/j.jssc.2010.01.006
2,Ba2YCu3O6,Y2O3;CuO;BaO,1470.0,NaN,Y-Ba-Cu-O,10.1016/j.jssc.2010.01.006
3,YBa2Cu3O7,CuO;BaCO3;Y2O3,940.0,24.000,Y-Ba-Cu-O,10.1016/j.physc.2004.11.003
4,Y1-xCaxBa1.9Nd0.1Cu3O,Y2O3;CuO;Nd2O3;BaCO3;CaO,950.0,36.000,Y-Ba-Cu-O-*,10.1016/S0921-4534(03)01275-9
...,...,...,...,...,...,...
215,YBa2Cu3O7,CuO;Y2O3;BaCu3,920.0,12.000,Y-Ba-Cu-O,10.1016/s0167-577x(02)00795-4
216,Y1Ba2Cu3O,CuO;BaCO3;Y2O3,945.0,16.000,Y-Ba-Cu-O,10.1016/s0925-8388(99)00115-2
217,(Y2-xSmx)Ba(Cu1-yCoy)O5,Y2O3;CuO;Sm2O3;CoO;BaCO3,850.0,32.000,Y-Ba-Cu-O-*,10.1016/s0955-2219(03)00179-1
218,Ba(NdxY2-x)CuO5,CuO;BaCO3;Nd2O3;Y2O3,980.0,23.875,Y-Ba-Cu-O-*,10.1016/j.jssc.2008.08.002


## Retrieve publication info from crossref

In [223]:
def get_cross_ref(ybco_data_path: str, email: str):
    '''
    Retrieve publication info from crossref and save.

    Args:
        ybco_data_path (str): Path to the YBCO dataframe
        email (str): Your email
    
    Returns:
        None
    '''
    df = pd.read_excel(ybco_data_path)
    dois = set()
    works_dict = {}
    for i, row in df.iterrows():
        doi = row['doi']
        dois.add(doi)
    cr = Crossref(mailto=email)
    responses = cr.works(ids=list(dois), progress_bar=True, warn=True)
    for x in responses:
        doi = x['message']['DOI']
        works_dict[doi] = x
    with open("data/crossref_works_ybco.json", 'w') as f:
        json.dump(works_dict, f)

In [224]:
# change this if you want to make the plot with the "pulled" dataset
# also add your email to be placed in the 'polite' pool at Crossref
ybco_data_path = "data/ybco_curated.xlsx"
email = ""
get_cross_ref(ybco_data_path, email)

## Prepare inputs and plot

In [236]:
def get_year_from_works(doi: str, works: dict) -> str:
    '''
    Returns the year from the crossref works given a doi.
    
    Args:
        doi (str): The doi for the recipe
        works (str): The works message from crossref
    
    Returns:
        year (str): The year the publciation was published
    '''
    dps = works[doi]['message']['published']['date-parts'][0]
    if len(dps) == 3:
        year, month, day = dps
    elif len(dps) == 2:
        year, month = dps
        day = None
    elif len(dps) == 1:
        year = dps
        month, day = None, None
    else:
        year, month, day = [None]*3
    if isinstance(year, list):
        year = year[0]
    return year

def resolve_years(df: pd.DataFrame, cross_ref_path: str) -> pd.DataFrame:
    '''
    Driver function for adding the year to the dataframe

    Args:
        df (pd.DataFrame): Dataframe of YBCO recipes
        cross_ref_path (str): The path to the retrived crossref works
    
    Returns:
        df (pd.DataFrame): Altered dataframe with year info
    '''
    with open(cross_ref_path) as f:
        works = json.load(f)
    years = []
    for i, row in df.iterrows():
        doi = row['doi']
        doi = doi.lower()
        year = get_year_from_works(doi, works)
        years.append(year)
    df['year'] = years
    return df

def resolve_ba_source(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Determines the ba-source precursor

    Args:
        df (pd.DataFrame): The YBCO dataframe
    
    Returns:
        df (pd.DataFrame): The YBCO dataframe with the ba-source resolved
    '''
    bas = []
    for i, row in df.iterrows():
        precursors = row['precursors']
        ps = precursors.split(';')
        count_ba = 0
        bs = None
        for p in ps:
            if 'Ba' in p:
                bs = p
                count_ba += 1
            if count_ba > 1:
                # don't know source. confusing
                bs = None
        if isinstance(bs, str):
            bs = bs.strip()
            # remove bad sources
            if 'x' in bs:
                bs = None
        if bs == 'BaCuO2(011)':
            bs = bs[:-5]
        bas.append(bs)
    df['ba-source'] = bas
    return df

def pretty_chemform(formula: str) -> str:
    '''
    Tries to put the chemical formula in tex output using simple logic

    Args:
        formual (str): The chemical formula
    
    Returns:
        ret (str): The tex version of the chemical formula
    '''
    ret_form = copy.copy(formula)
    counter = 0
    for i, c in enumerate(formula):
        if c.isnumeric() and formula[i-1] != '.':
            loc = i + counter
            ret_form = ret_form[:loc] + '_' + ret_form[loc:]
            counter += 1
    ret = rf"$\mathregular{{{ret_form}}}$"
    return ret

def make_save_scat(df: pd.DataFrame, save_path: str):
    '''
    Makes the scatter plot part of the figure

    Args:
        df (pd.DataFrame): The YBCO dataframe
        save_path (str): The save path for the figure
    
    Returns:
        None
    '''
    df = df.dropna()
    df['pretty-ba-source'] = df['ba-source'].apply(pretty_chemform)
    df = df.rename(columns={'pretty-ba-source': "Ba-source precursor", "year": "Publication year"})
    most_frequent = df["Ba-source precursor"].value_counts().reset_index()
    most_frequent.columns = ['Value', 'Frequency']
    most_frequent = most_frequent.sort_values(by='Frequency', ascending=False)
    ba_freq = list(most_frequent['Value'])
    plt.figure(figsize=(12, 8))
    ax = sns.scatterplot(data=df, y="t (hr)", x='T (C)', hue='Publication year', style="Ba-source precursor", palette=sns.cubehelix_palette(as_cmap=True), style_order=ba_freq)
    
    tmin = 0 
    tmax = 200
    Tmin = 400
    Tmax = 1600
    plt.ylim([0, 200])
    plt.xlim([400, 1600])
    plt.xlabel("Temperature (˚C)")
    plt.ylabel("Time (hours)")
    # EXTRACT CURRENT HANDLES AND LABELS
    h,l = ax.get_legend_handles_labels()

    # YEAR LEGEND
    year_leg = plt.legend(h[:6], l[:6], loc='upper right', 
                        bbox_to_anchor=(1, 1), fancybox=True, shadow=True)

    # PRECUR LEGEND
    precur_leg = plt.legend(h[-9:], l[-9:], loc='upper left', borderpad=1.6,
                        bbox_to_anchor=(0, 1), fancybox=True, shadow=True)

    # ADD FORMER (OVERWRITTEN BY LATTER)
    plt.gca().add_artist(precur_leg)
    plt.gca().add_artist(year_leg)
    #plt.savefig(f"{save_path}.svg", bbox_inches='tight')
    plt.savefig(f"{save_path}.png", bbox_inches='tight')
    plt.close()

def make_save_pies(df: pd.DataFrame, save_dir: str):
    '''
    Makes the pie charts part of the visualization. Some tex chemical formulas
    may be formatted poorly

    Args:
        df (pd.DataFrame): The YBCO dataframe
        save_dir (str): The directory to save the visualziation (e.g. figs)

    Returns:
        None
    '''
    # only drop the rows that lack a ba-source
    df = df.dropna(subset=['ba-source'])
    df['pretty-ba-source'] = df['ba-source'].apply(pretty_chemform)
    df = df.rename(columns={'pretty-ba-source': "Ba-source precursor", "year": "Publication year"})
    most_frequent = df["Ba-source precursor"].value_counts().reset_index()
    most_frequent.columns = ['Value', 'Frequency']
    most_frequent = most_frequent.sort_values(by='Frequency', ascending=False)
    # Data to plot
    labels = most_frequent['Value'].iloc[0], 'Other'
    sizes = [most_frequent['Frequency'].iloc[0]/most_frequent['Frequency'].sum(), sum(most_frequent['Frequency'].iloc[1:])/most_frequent['Frequency'].sum()]
    colors = ['#A5D7D2', '#FFC2B3']
    explode = (0.1, 0)  # explode 1st slice
    # Plot
    plt.pie(sizes, colors=colors)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.savefig(f"{save_dir}/small_pie.png")
    plt.close()

    def func(pct, allvalues):
        absolute = int(pct/100.*sum(allvalues))
        return ""
        return f"{absolute}"
    # Data to plot
    labels = list(most_frequent['Value'].iloc[1:])
    sizes = list(most_frequent['Frequency'].iloc[1:])
    #colors = ['#FF9999','#66B3FF','#99FF99','#FFCC99','#C2C2F0','#FFB3E6', '#C2F0C2', '#F0C2C2']
    # Plot
    patches, texts, autotexts = plt.pie(sizes, autopct=lambda pct: func(pct, sizes), startangle=90)
    for i, text in enumerate(autotexts):
        x, y = text.get_position()
        plt.text(x, y, sizes[i], ha='center', va='center', color='black')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.savefig(f"{save_dir}/big_pie.png")
    plt.close()

In [235]:
# data processing
ybco_path = "data/ybco_curated.xlsx"
cr_works_path = "data/crossref_works_ybco.json"
df = pd.read_excel(ybco_path)
df = resolve_years(df, cr_works_path)
df = resolve_ba_source(df)
df.to_excel("data/ybco_plot.xlsx")
# plotting
fig_save_path = "figs/ybco_scatter"
make_save_scat(df, fig_save_path)
make_save_pies(df, "figs")

                           Value  Frequency
0         $\mathregular{BaCO_3}$        150
1            $\mathregular{BaO}$          8
2     $\mathregular{Ba(NO_3)_2}$          6
3  $\mathregular{Ba(CH_3COO)_2}$          4
4          $\mathregular{BaO_2}$          4
5       $\mathregular{Ba_2CO_3}$          3
6        $\mathregular{BaCuO_2}$          1
7         $\mathregular{BaCu_3}$          1
                      target                    precursors   T (C)  t (hr)  \
0                YBa2Cu3O7-x              BaCO3; CuO; Y2O3   980.0    25.0   
1                YBa2Cu3O7-x              BaCO3; CuO; Y2O3     NaN     NaN   
2               Y1.5Ba2Cu3Ox               Y2O3; YBa2Cu3O7     NaN     NaN   
3                YBa2Cu3O7-x              BaCO3; CuO; Y2O3   938.0    24.0   
4                   Y2BaCuO5              BaCO3; CuO; Y2O3  1025.0    77.0   
..                       ...                           ...     ...     ...   
231    Fe0.5Cu0.5Ba2YCu2O7+x       BaCO3; CuO; Fe2O3; Y2O3

# Figure 5: Ternary compound convex hull

The process below will not recreate the figure in the publication exactly, but is a general strategy for producing ternary compound convex hulls

In [187]:
from pymatgen.analysis.phase_diagram import CompoundPhaseDiagram, PDPlotter
import pymatgen.core as pmg
from mp_api.client import MPRester

# Supply your MP key below
MP_KEY = ""

In [ ]:
# Get all the entries
with MPRester(MP_KEY) as mpr:
    entries = mpr.get_entries_in_chemsys("Sr-Fe-Mo-O")

In [208]:
pdgram = CompoundPhaseDiagram(
    entries,
    terminal_compositions=[
        pmg.Composition("SrO"),
        pmg.Composition("Fe2O3"),
        pmg.Composition("MoO3"),
    ],
)
# only show stable entries
fig = PDPlotter(pdgram, show_unstable=0.001).get_plot()
fig.show()